# Datalab 2025-2026

<div style="line-height:1;">

## Analyse over elektricitiets gebruik in de Nederlandse markt.

**Groepsnummer: 5**

**Auteurs: Lucas Hoetink, Thomas Galais, Mehmet Kilinc, Joris Zondervan**

<div style="line-height:1.6;">

## **Kernpuntenrapport**

Dit notebook presenteert doormiddel van data van het **CBS** een analytisch overzicht van de **Nederlandse energiemarkt**. Na het opschonen en voorbewerken van de rauwe datasets zijn er verschillende analyses op uitgevoerd om inzicht te krijgen in hoe het elekriciteits gebruik verschilt per gemeente en welke factoren dit beïnvloeden. 

Wij hebben datasets van het CBS doormiddel van <code style="color:#4EC9B0;">pandas</code>, <code style="color:#9CDCFE;">numpy</code> onderzocht. Ook hebben wij met <code style="color:#CE9178;">seaborn</code> en <code style="color:#CE9178;">matplotlib</code> meerdere visualisaties gemaakt om de data gemakkelijk in te zien.

**Belangrijkste bevindingen**:

- 
- 

Deze patronen benadrukken dit en dit en dit


---

## **Importeer de benodigde libraries**

Hier importeren wij <code style="color:#4EC9B0;">pandas</code> en <code style="color:#9CDCFE;">numpy</code>. Ook installeren wij eerst de api voor  <code style="color:#4EC9B0;">cbsodata</code> (CBS open data), en importeren we de library in het notebook.

In [3]:
import pandas as pd
import numpy as np
%pip install cbsodata
import cbsodata
import seaborn as sns
import matplotlib as plt

Note: you may need to restart the kernel to use updated packages.


## **Stel de plotting stijl in**

Om de uniformiteit te bewaren in het hele document gaan we alle grafieken dezelfde stijl geven.

In [4]:
plt.style.use('seaborn-v0_8')
sns.set_palette('husl')

---

## **Sprint 3**

<div style="line-height:1.6;">

### **Kernpuntenrapport**

Dit hoofdstuk presenteert een analytisch overzicht van het elektriciteitsverbruik per gemeente. Ook gaat dit hoofdstuk in op de verschillende factoren die het elektriciteitsverbruik per huishouden beïnvloeden.Als laatste gaan wij aan de hand van kenmerken van een buurt voorspellingen doen over het energieverbruik.

**Belangrijkste bevindingen:**

- 
- 

Dit verteld ons dit dit en dit

---

## **Importeer de dataset**

<!-- Zet de regel afstand voor betere leesbaarheid -->
<div style="line-height:1.6;">

**Hoe we dit doen:**

We beginnen met <code style="color:#4EC9B0;">pandas</code> en `.get_data()` van <code style="color:#4EC9B0;">cbsodata</code> om het dataset 84799NED op te halen en meteen om te zetten in een <code style="color:#4EC9B0;">pandas DataFrame</code>. 
We gaan meteen door met data verkenning om de dataset beter te begrijpen. De eerste verkenning met `.head()` en `.shape` geeft ons een snel overzicht van de datastructuur.

De dataset heeft `17341` rijen en `119` kolommen.
`.head()` laat ons de bovenste 5 rijen zien. hieruit kunnen we opmaken waar de dataset over gaat. Het is een dataset met informatie over verschillende wijken in Nederland. De informatie bevat onderwerpen zoals leeftijd, partnerschap, afkomst, geboorte en sterfte totalen, huishoudens, en meer. Interessante kolommen voor ons zijn bijvoorbeeld buurten en informatie over energieverbruik.

In [5]:
# Dataset importeren. Omdat de dataset bovenaan wat extra rijen heeft slaan we die over.
df = pd.DataFrame(cbsodata.get_data('84799NED'))

# Dataset uitlezen
print('De dataset heeft de volgende vorm:', df.shape)
print('\nDe bovenste 5 rijen zijn:\n',df.head())

De dataset heeft de volgende vorm: (17341, 119)

De bovenste 5 rijen zijn:
    ID          WijkenEnBuurten                            Gemeentenaam_1  \
0   0                Nederland  Nederland                                  
1   1              Aa en Hunze  Aa en Hunze                                
2   2            Wijk 00 Annen  Aa en Hunze                                
3   3                    Annen  Aa en Hunze                                
4   4  Verspreide huizen Annen  Aa en Hunze                                

  SoortRegio_2  Codering_3 IndelingswijzigingWijkenEnBuurten_4  \
0   Land        NL00                                         .   
1   Gemeente    GM1680                                       .   
2   Wijk        WK168000                                     1   
3   Buurt       BU16800000                                   1   
4   Buurt       BU16800009                                   1   

   AantalInwoners_5  Mannen_6  Vrouwen_7  k_0Tot15Jaar_8  ...  \
0    

<!-- Zet de regel afstand voor betere leesbaarheid -->
<div style="line-height:1.6;">

## **1. De dataset filteren**

We beginnen met een gestructureerd overzicht te creëren van de data die voor ons interessant is. Dit doen wij door de dataset te filteren en om te zetten in de DataFrames `buurten`, `inwoners`, en `elektriciteitsgebruik`. Het is nodig om op alleen de buurten te filteren, omdat ons onderzoek alleen de informatie over de buurten vereist. De niet-relevante informatie hindert de snelheid van berekeningen en door deze weg te laten maken wij het onderzoek ook overzichtelijker.
De interessante informatie voor ons in deze dataset is:

- De wijken en buurten kolom, die geeft ons informatie over locatie.
- De gemeente kolom geeft ons informatie over mogelijke groeperingen van data per gemeente.
- Het regio_type kolom geeft de mogelijkheid te sorteren op soort regio.
- De elektriciteitsverbruik kolom geeft ons inzicht in de gebruikte elektriciteit.

In [43]:
# Dataframe buurten maken
#Thomas
print(df.columns.tolist())

buurten = df[['WijkenEnBuurten', 'Gemeentenaam_1', 'SoortRegio_2', 'Bevolkingsdichtheid_33', 'Woningvoorraad_34', 'GemiddeldeWOZWaardeVanWoningen_35', 'PercentageEengezinswoning_36', 'PercentageMeergezinswoning_37', 'PercentageBewoond_38', 'PercentageOnbewoond_39', 'GIHandelEnHoreca_94', 'AfstandTotHuisartsenpraktijk_106', 'AfstandTotGroteSupermarkt_107', 'AfstandTotKinderdagverblijf_108', 'AfstandTotSchool_109', 'OppervlakteLand_112', 'OppervlakteWater_113', 'MateVanStedelijkheid_116', 'Omgevingsadressendichtheid_117']].copy()
inwoners = df[['k_20HuishoudensMetHoogsteInkomen_77', 'HuishoudensMetEenLaagInkomen_78', 'HuishoudensMetKinderen_31','HuishoudensZonderKinderen_30', 'Bevolkingsdichtheid_33', 'OpleidingsniveauLaag_64', 'OpleidingsniveauMiddelbaar_65', 'OpleidingsniveauHoog_66', 'Mannen_6', 'Vrouwen_7','k_0Tot15Jaar_8', 'k_15Tot25Jaar_9', 'k_25Tot45Jaar_10', 'k_45Tot65Jaar_11', 'k_65JaarOfOuder_12','GeboorteTotaal_24', 'GeboorteRelatief_25', 'SterfteTotaal_26', 'SterfteRelatief_27',]]
elektriciteitsverbruik = df[['GemiddeldElektriciteitsverbruikTotaal_47', 'GemiddeldAardgasverbruikTotaal_55', 'BouwjaarVoor2000_45', 'BouwjaarVanaf2000_46','PercentageWoningenMetStadsverwarming_63',]]
woningen = df[['GemiddeldeHuishoudensgrootte_32', 'Appartement_48', 'Tussenwoning_49', 'Hoekwoning_50','TweeOnderEenKapWoning_51', 'VrijstaandeWoning_52', 'BedrijfsvestigingenTotaal_91',]]

buurten.rename(columns={'WijkenEnBuurten': 'buurt', 'Gemeentenaam_1': 'gemeente', 'SoortRegio_2': 'regio_type', 'GemiddeldElektriciteitsverbruikTotaal_47':'gem_elek_verbruik'}, inplace=True)



# Een nieuwe dataframe met alleen buurten is belangrijk omdat we dan alle "onnodige" informatie buiten laten en alle bruikbare inforamtie voor ons onderzoek overhouden.

['ID', 'WijkenEnBuurten', 'Gemeentenaam_1', 'SoortRegio_2', 'Codering_3', 'IndelingswijzigingWijkenEnBuurten_4', 'AantalInwoners_5', 'Mannen_6', 'Vrouwen_7', 'k_0Tot15Jaar_8', 'k_15Tot25Jaar_9', 'k_25Tot45Jaar_10', 'k_45Tot65Jaar_11', 'k_65JaarOfOuder_12', 'Ongehuwd_13', 'Gehuwd_14', 'Gescheiden_15', 'Verweduwd_16', 'WestersTotaal_17', 'NietWestersTotaal_18', 'Marokko_19', 'NederlandseAntillenEnAruba_20', 'Suriname_21', 'Turkije_22', 'OverigNietWesters_23', 'GeboorteTotaal_24', 'GeboorteRelatief_25', 'SterfteTotaal_26', 'SterfteRelatief_27', 'HuishoudensTotaal_28', 'Eenpersoonshuishoudens_29', 'HuishoudensZonderKinderen_30', 'HuishoudensMetKinderen_31', 'GemiddeldeHuishoudensgrootte_32', 'Bevolkingsdichtheid_33', 'Woningvoorraad_34', 'GemiddeldeWOZWaardeVanWoningen_35', 'PercentageEengezinswoning_36', 'PercentageMeergezinswoning_37', 'PercentageBewoond_38', 'PercentageOnbewoond_39', 'Koopwoningen_40', 'HuurwoningenTotaal_41', 'InBezitWoningcorporatie_42', 'InBezitOverigeVerhuurders_43'

---

<!-- Zet de regel afstand voor betere leesbaarheid -->
<div style="line-height:1.6;">

## **2. Dataset inspecteren**

Nu dat wij de dataset hebben gefiltered kunnen we kijken naar de interessante informatie. We beginnen met een gestructueerd overzicht via de `.info()` methode. Dit geeft ons het aantal rijen, kolommen, de hoeveelheid bestaande waarden, en de kolom namen. Dit is essentieel voor het begrijpen en schoonmaken van onze data.

In [ ]:
# Aantal rijen en kolommen dataframe
df.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17341 entries, 0 to 17340
Columns: 119 entries, ID to Omgevingsadressendichtheid_117
dtypes: float64(75), int64(38), object(6)
memory usage: 15.7+ MB


<!-- Zet de regel afstand voor betere leesbaarheid -->
<div style="line-height:1.6;">

#### **2.1 Omschrijving van de kolommen**

Onze dataset heeft X aantal rijen en X aantal kolommen. De kolommen hebben wij hieronder beschreven met informatie over wat de kolom is, wat voor type data er in zit, welke eenheden de data in is, en of er missende waarden zijn.


| kolomnaam | omschrijving | type | eenheden | missende waarden |
| ---       |---           | ---  | ---      | ---              |
| ...       | ...  | ...      | ...              |...

#### **2.2 Extra omschrijving numerieke kolommen**

Om nog beter inzicht in de data te krijgen en ook om de data in perspectief te zetten hebben we hieronder laten zien wat de maximale, minimale, gemiddelde, en mediane waardes zijn in alle kolommen. Een grote afwijking van mediaan en gemiddelde komt vaak door uitschieters zoals we kunne zien in kolommen X en X.


| kolomnaam | maximum | minimum | gemiddelde | mediaan |
|---        | ---     |---      |---         |---      |
|...        | ...     |...      |...         |...      |



<!-- Zet de regel afstand voor betere leesbaarheid -->
<div style="line-height:1.6;">

#### **2.3 Relevante kolommen**

Niet alle kolommen van de dataset zijn interessant voor het onderzoek. Wij kunnen van sommige kolommen nu al zeggen dat deze niet relevant zijn. Daarom gaan wij in de volgende stap de kolommen die wij niet nodig hebben verwijderen. Dit zorgt voor een schonere dataset die overzichtelijker is en waar wij makkelijker inzichten uit kunnen krijgen.

De irrelevante kolommen die wij verwijderd hebben zijn:
<!-- en waarom -->
- 
- 
- 

De kolommen die wij houden en die dus wel interessant zijn zijn de volgende:
<!-- en waarom -->
- 
- 
- 

In [ ]:
# Verwijderen van irrelevante kolommen

<!-- Zet de regel afstand voor betere leesbaarheid -->
<div style="line-height:1.6;">

#### **2.4 Onderbouwing te veel missende waarden**

Sommige kolommen hebben (te) veel missende waarden. Deze kolommen moeten ook verwijderd worden om de betrouwbaarheid van de data hoog te houden. Kolommen met X% aantal missende waarden moeten worden verwijdert. Deze marge hebben wij genomen omdat XXXXXX.

De kolommen X X en X <!-- placeholder --> hebben te veel missende waarden en zullen moeten worden verwijderd. Dit doen wij met de `XXX` methode/functie.

In [ ]:
# Verwijderen van kolommen met te veel missende waarden

<!-- Zet de regel afstand voor betere leesbaarheid -->
<div style="line-height:1.6;">

#### **2.5 Groeperen kolommen**

Om de dataset beter de begrijpen kunnen wij sommige kolommen, zolas *"Woningtype"* groeperen. Dit helpt ons door meer perspectief te kunnnen geven in de data doormiddel van gegroepeerde statestieken en door data analyse later makkelijker en duidelijker te maken.

Wij maken hier de groepen X X en X aan doormiddel van de `.groupby()` methode uit <code style="color:#4EC9B0;">pandas</code>.
Daarna kunnen wij doormiddel van <code style="color:#CE9178;">seaborn</code> en <code style="color:#CE9178;">matplotlib</code> wat visuele overzichten creëeren om meer inzicht te creëeren in de data.

In [ ]:
# nieuwe gegroepeerde datasets

<!-- Zet de regel afstand voor betere leesbaarheid -->
<div style="line-height:1.6;">

#### **2.6 Basisstatistieken per gemeente**

Om de data verder te verkennen gaan we nu wat meer basisstatistieken opzoeken over de gemeentes. We gaan de totalen, gemiddleden, en medianen berekenen doormiddel van <code style="color:#9CDCFE;"> numpy</code>. Deze geven ons inzicht in verschillen tussen gemeenten. Daaruit kunnen wij conclusies of voorspellingen halen.

Wij hebben de volgende ondervindingen gedaan:

- 
- 
- 
- 

In [ ]:
# Basisstatistieken:
# Totalen per gemeente

<!-- Zet de regel afstand voor betere leesbaarheid -->
<div style="line-height:1.6;">

Om onze data te verifieren hebben wij onze ondervindingen vergelijken met de data op de CBS website. Hier hebben wij WEL/NIET een verschil in gemerkt. Namelijk het volgende... Dit is te verklaren door dit dit en dit.

#### **2.7 Basistatastieken percentages**

Wij kunnen nog meer inzichten halen uit de kolommen met aantallen. Wij hebben om beter inzicht te krijgen in X en X de volgende percentages uitgerekend:

- 
- 
- 

Dit heeft ons de volgende inzichten gegeven: XXXXX

In [ ]:
# Basisstatistieken:
# Percentages

---

<!-- Zet de regel afstand voor betere leesbaarheid -->
<div style="line-height:1.6;">

## 3. **Voorlopige conclusie**

Deze analyse heeft duidelijke patronen bevonden op dit dit en dit vlak. De gemeenten zouden dit dit en dit kunnen doen om de druk op het electriciteitsnet te verlagen.

**Belangrijkste prestaties:**

<!-- Placeholders -->

- Er zijn XXXX observaties verwerkt, waarmee 95,0% van de oorspronkelijke dataset is behouden.
- Geen enkele gemeente doet iets heel ergs
- Op weekdagen werd een stijging van zoveel energie gemeten
- Er zijn 69 gemeenten zijn in kaart gebracht, waarbij hoogrisicogebieden en verschillen tussen provincies zijn geïdentificeerd.

---

## Sprint 4

## Sprint 5

## Sprint 6

### Eindadvies